In [1]:
import ast
import json
import pickle
import pandas as pd
import numpy as np

from glob import glob
from mlblocks import MLPipeline
from orion.data import load_signal
from orion.analysis import _build_events_df
from orion.evaluation import contextual_f1_score

from sintel.actions import annotator, add_label

In [14]:
np.random.seed(0)

DATA_PATH = '../sintel/data/{}'

UNSUPERVISED_PIPELINE_DIR = '../benchmark/pipelines/{}'

ANOMALIES = pd.read_csv(DATA_PATH.format('anomalies.csv'))

UNSUPERVISED_PIPELINE = glob(UNSUPERVISED_PIPELINE_DIR.format('*.pkl'))

BENCHMARK_DATA = pd.read_csv(
    DATA_PATH.format('datasets.csv'), index_col=0, header=None).applymap(ast.literal_eval).to_dict()[1]


INTERVAL = {
    "realTweets": 300
}

INTERVAL_FMT = {
    "realTweets": 3600
}

In [3]:
def load_anomalies(signal):
    anomalies = ANOMALIES.set_index('signal').loc[signal].values[0]
    anomalies = pd.DataFrame(json.loads(anomalies), columns=['start', 'end'])
    return anomalies

def _build_events(events):
    events = pd.DataFrame(list(events), columns=['start', 'end'])
    events['start'] = events['start'].astype(int)
    events['end'] = events['end'].astype(int)

    return events


def _merge_sequences(sequences):
    if len(sequences) == 0:
        return np.array([])
    
    if not isinstance(sequences, list):
        sequences = list(sequences[['start', 'end']].itertuples(index=False))

    sorted_sequences = sorted(sequences, key=lambda entry: entry[0])
    new_sequences = [sorted_sequences[0]]
    weights = [sorted_sequences[0][1] - sorted_sequences[0][0]]

    for sequence in sorted_sequences[1:]:
        prev_sequence = new_sequences[-1]

        if sequence[0] <= prev_sequence[1] + 1:
            weights.append(sequence[1] - sequence[0])
            new_sequences[-1] = (prev_sequence[0], max(prev_sequence[1], sequence[1]))

        else:
            weights = [sequence[1] - sequence[0]]
            new_sequences.append(sequence)

    return np.array(new_sequences)

def _merge_events(first, second):
    first = first.copy()[['start', 'end']]
    second = second.copy()[['start', 'end']]
    
    events = pd.concat([first, second])
    return _merge_sequences(events)

### feedback experiment process

1. use `lstm_dynamic_threshold` to find anomalies using unsupervised learning.
2. use `simulation` to select a number of fp, fn to correct by the simulator in the train dataset:
    - this selection can be weighted by severity score
    - this selection can be random
3. train the `lstm_supervised` pipeline on the annotated timeseries
4. evaluate the performance on the test dataset.
5. repeat step 2 - 4.

**excluding**
NASA dataset due to the fact that nasa dataset is a pre-split into (train: no anomalies, test: with anomalies), therefore, the simulation will need to be executed on the test dataset which is illogical.

In [4]:
def get_anomalies_in_test(anomalies, start):
    df = anomalies.copy()
    remove = []
    for i, anom in df.iterrows():
        if anom.start < start and anom.end > start:
            df.at[i, 'start'] = start
        elif anom.start < start:
            remove.append(i)

    return df.drop(remove)

def get_anomalies_in_train(anomalies, start):
    df = anomalies.copy()
    remove = []
    for i, anom in df.iterrows():
        if anom.start < start and anom.end > start:
            df.at[i, 'end'] = start
        elif anom.start > start:
            remove.append(i)

    return df.drop(remove)

In [5]:
dataset = 'realTweets'

for signal in BENCHMARK_DATA[dataset]:
    
# data
signal = 'Twitter_volume_AAPL'

data_path = DATA_PATH.format(signal + '.csv')
timeseries = load_signal(data_path)
train, test = load_signal(data_path, test_size=0.3)
ground_truth = load_anomalies(signal)

# train
train['label'] = [0] * len(train)
train = train.set_index('timestamp').sort_index()  
train = train.reset_index()

# test
test['label'] = [0] * len(test)
test = test.set_index('timestamp').sort_index()  
test = test.reset_index()

# unsupervised

pipeline_path = [x for x in UNSUPERVISED_PIPELINE if signal in x][0]

with open(pipeline_path, 'rb') as pfile:
    pipeline = pickle.load(pfile)
    
anomalies = pipeline.predict(timeseries)
anomalies = _build_events_df(anomalies)

start = test['timestamp'].min()

# train 
train_anomalies = get_anomalies_in_train(anomalies, start)
train_ground_truth = get_anomalies_in_train(ground_truth, start)

# test 
test_anomalies = get_anomalies_in_test(anomalies, start)
test_ground_truth = get_anomalies_in_test(ground_truth, start)

class_weights = {0: 1, 1: 1e3}
interval = INTERVAL[dataset]
interval_fmt = INTERVAL_FMT[dataset]

hyperparameters = {
    "mlprimitives.custom.timeseries_preprocessing.time_segments_aggregate#1": {
        "interval": interval,
    },
    "keras.Sequential.LSTMTimeSeriesClassifier#1": {
        "epochs": 10,
        'validation_split': 0.0,
        'verbose': False
    },
    "sintel.primitives.timeseries_anomalies.format_anomalies#1": {
        "interval": interval_fmt
    }
}

supervised_pipeline = MLPipeline("lstm_supervised")
supervised_pipeline.set_hyperparameters(hyperparameters)

# simulate annotation

k = 1
events = _merge_sequences(annotator(train_anomalies, train_ground_truth, k))
stop = contextual_f1_score(events, train_ground_truth, weighted=False) == 1.0

while not stop:

    train = train.set_index('timestamp').sort_index()
    for a in events:
        train.at[a.start: a.end, 'label'] = 1

    train = train.reset_index()
    train.head()

    supervised_pipeline.fit(train, class_weights=class_weights)

    # train score
    train_anomalies = supervised_pipeline.predict(train)
    train_anomalies = _build_events(train_anomalies)

    train_f1_score = contextual_f1_score(train_anomalies, train_ground_truth, weighted=False)

    # test score
    test_anomalies = supervised_pipeline.predict(test)
    test_anomalies = _build_events(test_anomalies)

    test_f1_score = contextual_f1_score(test_anomalies, test_ground_truth, weighted=False)
    
    # display
    print("train score: ", train_f1_score)
    print("test score: ", test_f1_score)
    
    events = _merge_sequences(annotator(train_anomalies, train_ground_truth, k))
    stop = contextual_f1_score(events, train_ground_truth, weighted=False) == 1.0